In [364]:
import sys
print(sys.version)

3.13.1 (main, Dec  3 2024, 17:59:52) [Clang 16.0.0 (clang-1600.0.26.4)]


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [3]:
    spark = SparkSession \
        .builder \
        .appName("CSV Join Demo") \
        .master("local[3]") \
        .getOrCreate()
spark.sparkContext.setLogLevel("OFF")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/03 12:41:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.format("csv").option("header", "true").load("/Users/akhitha/pycharmProjects/Spark-Programming-In-Python-master/playground/data/olympics/Teams.csv")


In [7]:

basepath ="/Users/akhitha/pycharmProjects/Spark-Programming-In-Python-master/playground/data/olympics"

# Athletes_DF = spark.read.format("csv").option("header", "true").load(f"{basepath}/Athletes.csv")
# Coaches_DF = spark.read.format("csv").option("header", "true").load(f"{basepath}/Coaches.csv")
# EntriesGender_DF = spark.read.format("csv").option("header", "true").load(f"{basepath}/EntriesGender.csv")
# Medals_DF = spark.read.format("csv").option("header", "true").load(f"{basepath}/Medals.csv")
# Teams_DF = spark.read.format("csv").option("header", "true").load(f"{basepath}/Teams.csv")


athletesDf = spark.read.format("csv").option("header", "true").load(f"{basepath}/Athletes.csv")
CoachesDf = spark.read.format("csv").option("header", "true").load(f"{basepath}/Coaches.csv")
entriesGenderDf = spark.read.format("csv").option("header", "true").load(f"{basepath}/EntriesGender.csv")
medalsDf = spark.read.format("csv").option("header", "true").load(f"{basepath}/Medals.csv")
teamsDf = spark.read.format("csv").option("header", "true").load(f"{basepath}/Teams.csv")



In [8]:
print("scenario: 1.Find the total number of male and female entries for each discipline where Japan has coaches")

GenderCountDf= CoachesDf.join(entriesGenderDf, on="Discipline", how="right")\
.filter(col("Country") == "Japan")  \
.groupBy("Discipline") \
.agg(sum("Male").alias("MaleCount"), sum("Female").alias("FemaleCount")) \
.select("Discipline","MaleCount","FemaleCount")

GenderCountDf.show()

scenario: 1.Find the total number of male and female entries for each discipline where Japan has coaches


[Stage 13:>                                                         (0 + 1) / 1]

+-----------------+---------+-----------+
|       Discipline|MaleCount|FemaleCount|
+-----------------+---------+-----------+
|Baseball/Softball|    576.0|      360.0|
|       Basketball|    720.0|      720.0|
|         Handball|    504.0|      504.0|
|     Rugby Sevens|    453.0|      438.0|
|       Water Polo|    292.0|      244.0|
|       Volleyball|    720.0|      720.0|
|Artistic Swimming|      0.0|      420.0|
|           Hockey|    576.0|      576.0|
|         Football|   2064.0|     1584.0|
+-----------------+---------+-----------+



In [9]:
print("scenario: 1.Find the total number of male and female entries for each discipline where Japan has coaches")

GenderCountDf= CoachesDf.join(entriesGenderDf, on="Discipline", how="inner")\
.filter(col("Country") == "Japan")  \
.groupBy("Discipline") \
.agg(sum("Male").alias("MaleCount"), sum("Female").alias("FemaleCount")) \
.select("Discipline","MaleCount","FemaleCount")

GenderCountDf.show()

scenario: 1.Find the total number of male and female entries for each discipline where Japan has coaches
+-----------------+---------+-----------+
|       Discipline|MaleCount|FemaleCount|
+-----------------+---------+-----------+
|Baseball/Softball|    576.0|      360.0|
|       Basketball|    720.0|      720.0|
|         Handball|    504.0|      504.0|
|     Rugby Sevens|    453.0|      438.0|
|       Water Polo|    292.0|      244.0|
|       Volleyball|    720.0|      720.0|
|Artistic Swimming|      0.0|      420.0|
|           Hockey|    576.0|      576.0|
|         Football|   2064.0|     1584.0|
+-----------------+---------+-----------+



In [113]:
medalsDf.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [115]:
teamsDf.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [17]:
    print("Scenario: 2.Determine which country has the highest average number of medals per discipline")
    totalmedals_df=medalsDf.groupBy("TeamCountry").agg(sum("Total").alias("totalMedals"))
    totalmedals_df.show()


Scenario: 2.Determine which country has the highest average number of medals per discipline
+--------------------+-----------+
|         TeamCountry|totalMedals|
+--------------------+-----------+
|       Côte d'Ivoire|        1.0|
|Islamic Republic ...|        7.0|
|      Chinese Taipei|       12.0|
|              Sweden|        9.0|
|   Republic of Korea|       20.0|
|         Philippines|        4.0|
|            Malaysia|        2.0|
|                Fiji|        2.0|
|              Turkey|       13.0|
|             Germany|       37.0|
|              Jordan|        2.0|
|              France|       33.0|
|              Greece|        4.0|
|              Kosovo|        2.0|
|            Slovakia|        4.0|
|    Hong Kong, China|        6.0|
|           Argentina|        3.0|
|             Belgium|        7.0|
|       Great Britain|       65.0|
|          San Marino|        3.0|
+--------------------+-----------+
only showing top 20 rows



In [18]:
    print("counting discipline")
    DisciplineCountDF=teamsDf.select("Country","Discipline").groupBy("Discipline","country").agg(count("Discipline").alias("DisciplineCount"))
    DisciplineCountDF.show()


counting discipline
+-------------------+--------------------+---------------+
|         Discipline|             country|DisciplineCount|
+-------------------+--------------------+---------------+
|Artistic Gymnastics|               Italy|              1|
|      Cycling Track|         Switzerland|              2|
|Artistic Gymnastics|       Great Britain|              2|
|         Water Polo|                 ROC|              1|
|            Archery|             Ukraine|              2|
|  Artistic Swimming|             Belarus|              1|
|      Cycling Track|              Canada|              4|
|      Cycling Track|             Ireland|              2|
|  Artistic Swimming|               Japan|              2|
|      Cycling Track|                 ROC|              3|
|       Table Tennis|             Germany|              3|
|      Cycling Track|         New Zealand|              5|
|   Beach Volleyball|              Brazil|              4|
|      Cycling Track|             De

In [19]:
joined_df=totalmedals_df.join(DisciplineCountDF,totalmedals_df.TeamCountry ==DisciplineCountDF["country"], how="inner")
avg_medals_df= joined_df.withColumn("avg_medals_per_descipline", col("totalMedals")/col("DisciplineCount"))
joined_df.show()
avg_medals_df.show()

+--------------------+-----------+-------------------+--------------------+---------------+
|         TeamCountry|totalMedals|         Discipline|             country|DisciplineCount|
+--------------------+-----------+-------------------+--------------------+---------------+
|               Italy|       40.0|Artistic Gymnastics|               Italy|              1|
|         Switzerland|       13.0|      Cycling Track|         Switzerland|              2|
|       Great Britain|       65.0|Artistic Gymnastics|       Great Britain|              2|
|                 ROC|       71.0|         Water Polo|                 ROC|              1|
|             Ukraine|       19.0|            Archery|             Ukraine|              2|
|             Belarus|        7.0|  Artistic Swimming|             Belarus|              1|
|              Canada|       24.0|      Cycling Track|              Canada|              4|
|             Ireland|        4.0|      Cycling Track|             Ireland|     

In [13]:
highest_avg_medal=avg_medals_df.orderBy(col("avg_medals_per_descipline").desc()).limit(1)
highest_avg_medal.show()


+--------------------+-----------+--------------+--------------------+---------------+-------------------------+
|         TeamCountry|totalMedals|    Discipline|             country|DisciplineCount|avg_medals_per_descipline|
+--------------------+-----------+--------------+--------------------+---------------+-------------------------+
|United States of ...|      113.0|3x3 Basketball|United States of ...|              1|                    113.0|
+--------------------+-----------+--------------+--------------------+---------------+-------------------------+



In [ ]:
3. Group By and Sorting

 • Scenario: List the top 3 countries with the highest number of gold medals in athletics.
 • Tasks:
 • Filter the Medals dataset for athletics (assume it’s included in Event for Teams).
 • Group by TeamCountry and sum the gold medals.
 • Sort in descending order and pick the top 3.

In [23]:
medalsDf.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [24]:
athletesDf.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [31]:
    sc3Df= medalsDf.join(athletesDf, medalsDf.TeamCountry==athletesDf.Country, "inner") \
                .groupBy("TeamCountry") \
            .agg(sum("gold").alias("totalGoldMedals")) \
            .orderBy(col("totalGoldMedals").desc()).limit(3)          
sc3Df.show()


+--------------------+---------------+
|         TeamCountry|totalGoldMedals|
+--------------------+---------------+
|United States of ...|        23985.0|
|               Japan|        15822.0|
|People's Republic...|        15238.0|
+--------------------+---------------+



In [ ]:
Scenario: Analyze gender participation in all disciplines.
 • Tasks:
 • Load the EntriesGender dataset.
 • Calculate the proportion of male and female participants for each discipline.
 • Identify disciplines with more than 75% female participants

In [32]:
entriesGenderDf.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [156]:
from pyspark.sql.functions import col, count, lit, concat, format_number,window


In [152]:
df=entriesGenderDf\
.withColumn("Femalepercentage_Long", (floor(col("FeMale").cast("double")/col("Total").cast("double")*100))) \
.filter(col("Femalepercentage_Long") < 50)\
.withColumn("Femalepercentage",concat(col("Femalepercentage_Long"),lit("%"))) \
.drop("Femalepercentage_Long")

df.show()

df.printSchema()

+-----------------+------+----+-----+----------------+
|       Discipline|Female|Male|Total|Femalepercentage|
+-----------------+------+----+-----+----------------+
|        Athletics|   969|1072| 2041|             47%|
|        Badminton|    86|  87|  173|             49%|
|Baseball/Softball|    90| 144|  234|             38%|
|           Boxing|   102| 187|  289|             35%|
|     Canoe Sprint|   123| 126|  249|             49%|
|     Cycling Road|    70| 131|  201|             34%|
|    Cycling Track|    90|  99|  189|             47%|
|       Equestrian|    73| 125|  198|             36%|
|          Fencing|   107| 108|  215|             49%|
|         Football|   264| 344|  608|             43%|
|             Judo|   192| 201|  393|             48%|
|           Karate|    40|  42|   82|             48%|
|           Rowing|   257| 265|  522|             49%|
|     Rugby Sevens|   146| 151|  297|             49%|
|         Swimming|   361| 418|  779|             46%|
|         

In [115]:
filteredDf= df.filter(col("Femalepercentage") < 75)
filteredDf.show()

+--------------------+------+----+-----+----------------+-----------------------------+
|          Discipline|Female|Male|Total|Femalepercentage|proportion_with_Femalepercent|
+--------------------+------+----+-----+----------------+-----------------------------+
|      3x3 Basketball|    32|  32|   64|              50|                       50.00%|
|             Archery|    64|  64|  128|              50|                       50.00%|
| Artistic Gymnastics|    98|  98|  196|              50|                       50.00%|
|           Athletics|   969|1072| 2041|              47|                       47.00%|
|           Badminton|    86|  87|  173|              49|                       49.00%|
|   Baseball/Softball|    90| 144|  234|              38|                       38.00%|
|          Basketball|   144| 144|  288|              50|                       50.00%|
|    Beach Volleyball|    48|  48|   96|              50|                       50.00%|
|              Boxing|   102| 18

In [58]:
    proportionDF = entriesGenderDf.withColumn("malepercrntage", floor(col("Male")/col("Total") *100)) \
        .withColumn("Femaleaddedpercentail", concat(col("Female"),lit("%"))) \
     .withColumn("Femalepercentage", (floor(col("Female")/col("Total")*100))) 

proportionDF.show()

+--------------------+------+----+-----+--------------+---------------------+----------------+
|          Discipline|Female|Male|Total|malepercrntage|Femaleaddedpercentail|Femalepercentage|
+--------------------+------+----+-----+--------------+---------------------+----------------+
|      3x3 Basketball|    32|  32|   64|            50|                  32%|              50|
|             Archery|    64|  64|  128|            50|                  64%|              50|
| Artistic Gymnastics|    98|  98|  196|            50|                  98%|              50|
|   Artistic Swimming|   105|   0|  105|             0|                 105%|             100|
|           Athletics|   969|1072| 2041|            52|                 969%|              47|
|           Badminton|    86|  87|  173|            50|                  86%|              49|
|   Baseball/Softball|    90| 144|  234|            61|                  90%|              38|
|          Basketball|   144| 144|  288|          

In [ ]:
 • Scenario:5 Add a ranking column to the Medals dataset based on the total number of medals.
 • Tasks:
 • Use the Rank by Total column to sort countries.
 • Add a dense rank column based on the total medal count.


In [169]:
medalsDf.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [172]:
from pyspark.sql.window import Window

rankDf=medalsDf.withColumn("ranking", dense_rank().over(Window.orderBy(col("Total").cast("int").desc()))) 
rankDf.show()


+----+--------------------+----+------+------+-----+-------------+-------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|ranking|
+----+--------------------+----+------+------+-----+-------------+-------+
|   1|United States of ...|  39|    41|    33|  113|            1|      1|
|   2|People's Republic...|  38|    32|    18|   88|            2|      2|
|   5|                 ROC|  20|    28|    23|   71|            3|      3|
|   4|       Great Britain|  22|    21|    22|   65|            4|      4|
|   3|               Japan|  27|    14|    17|   58|            5|      5|
|   6|           Australia|  17|     7|    22|   46|            6|      6|
|  10|               Italy|  10|    10|    20|   40|            7|      7|
|   9|             Germany|  10|    11|    16|   37|            8|      8|
|   7|         Netherlands|  10|    12|    14|   36|            9|      9|
|   8|              France|  10|    12|    11|   33|           10|     10|
|  11|              Canad

In [ ]:
Handling Complex Data

 • Scenario: Identify disciplines where coaches and athletes share the same country.
 • Tasks:
 • Join the Coaches and Athletes datasets on Discipline and Country.
 • List unique disciplines where both coaches and athletes are from the same country.


In [218]:
CoachesDf.show()
CoachesDf.printSchema()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [219]:
athletesDf.show()
athletesDf.printSchema()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [214]:
joinCADf= CoachesDf.join(athletesDf).where((CoachesDf.Country==athletesDf.Country) & (CoachesDf.Discipline==athletesDf.Discipline))

In [217]:
joinmultiDf=CoachesDf.join(athletesDf,["Country","Discipline"],"inner") \
.select("Discipline","Country")\
.distinct()
joinmultiDf.show()

+-----------------+------------------+
|       Discipline|           Country|
+-----------------+------------------+
|       Water Polo|               ROC|
|Artistic Swimming|           Belarus|
|Artistic Swimming|             Japan|
|       Volleyball|            Serbia|
|     Rugby Sevens|           Ireland|
|           Hockey|             Japan|
|         Football|          Honduras|
|Artistic Swimming|          Colombia|
|         Handball|           Denmark|
|       Volleyball| Republic of Korea|
|           Hockey|      South Africa|
|       Basketball|         Argentina|
|       Volleyball|Dominican Republic|
|         Handball| Republic of Korea|
|         Football|       New Zealand|
|     Rugby Sevens|            France|
|       Volleyball|            Brazil|
|         Football|             Chile|
|         Handball|            Angola|
|       Water Polo|             Japan|
+-----------------+------------------+
only showing top 20 rows



In [ ]:
7. Window Functions

 • Scenario: Find the most successful coach by event based on the number of medals won by their team.
 • Tasks:


In [226]:
teamsDf.show()
teamsDf.printSchema()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [224]:
medalsDf.show()
medalsDf.printSchema()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [269]:
renameTeamsDf=teamsDf.withColumnRenamed("Event","TeamsEvent")
renameTeamsDf.show()


+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|  TeamsEvent|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [280]:
join2Dfs=CoachesDf.join(renameTeamsDf,["Country","Discipline"],"inner")
join2Dfs.show()

+--------------------+-----------------+-------------------+-----+---------+----------+
|             Country|       Discipline|               Name|Event| TeamName|TeamsEvent|
+--------------------+-----------------+-------------------+-----+---------+----------+
|           Australia|Artistic Swimming|   MONTICO Loredana| Team|Australia|      Duet|
|           Australia|Artistic Swimming|   MONTICO Loredana| Duet|Australia|      Duet|
|           Australia|Artistic Swimming|   MONTICO Loredana| Team|Australia|      Team|
|           Australia|Artistic Swimming|   MONTICO Loredana| Duet|Australia|      Team|
|             Austria|Artistic Swimming|   SHAHNAZI Shahbaz| NULL|  Austria|      Duet|
|             Austria|Artistic Swimming|   MLADENOVA Albena| Duet|  Austria|      Duet|
|             Austria|Artistic Swimming|      BRUN Aristide| NULL|  Austria|      Duet|
|             Belarus|Artistic Swimming|SVETLICHNAYA Aliona| Duet|  Belarus|      Duet|
|              Canada|Artistic S

In [281]:
join3Dfs= join2Dfs.join(medalsDf,join2Dfs["Country"]==medalsDf["TeamCountry"],"inner")
join3Dfs.show(truncate=False)

+--------------------------+-----------------+-------------------+-----+---------+----------+----+--------------------------+----+------+------+-----+-------------+
|Country                   |Discipline       |Name               |Event|TeamName |TeamsEvent|Rank|TeamCountry               |Gold|Silver|Bronze|Total|Rank by Total|
+--------------------------+-----------------+-------------------+-----+---------+----------+----+--------------------------+----+------+------+-----+-------------+
|Australia                 |Artistic Swimming|MONTICO Loredana   |Team |Australia|Duet      |6   |Australia                 |17  |7     |22    |46   |6            |
|Australia                 |Artistic Swimming|MONTICO Loredana   |Duet |Australia|Duet      |6   |Australia                 |17  |7     |22    |46   |6            |
|Australia                 |Artistic Swimming|MONTICO Loredana   |Team |Australia|Team      |6   |Australia                 |17  |7     |22    |46   |6            |
|Australia

In [279]:

winspec=Window.partitionBy("Name").orderBy(col("Total").cast("int").desc())
winDf=join3Dfs.withColumn("rank", dense_rank().over(winspec)) \
.select("Name","TeamsEvent","Total","rank","Discipline","Teamname")
winDf.show()


+--------------------+----------+-----+----+-----------------+--------------------+
|                Name|TeamsEvent|Total|rank|       Discipline|            Teamname|
+--------------------+----------+-----+----+-----------------+--------------------+
|     ABDELMAGID Wael|       Men|    6|   1|         Football|               Egypt|
|           ABE Junya|       Men|   58|   1|       Volleyball|               Japan|
|           ABE Junya|     Women|   58|   1|       Volleyball|               Japan|
|       ABE Katsuhiko|       Men|   58|   1|       Basketball|               Japan|
|       ABE Katsuhiko|     Women|   58|   1|       Basketball|               Japan|
|          AGEBA Yuya|       Men|   58|   1|       Volleyball|               Japan|
|          AGEBA Yuya|     Women|   58|   1|       Volleyball|               Japan|
|AIKMAN Siegfried ...|       Men|   58|   1|           Hockey|               Japan|
|AIKMAN Siegfried ...|     Women|   58|   1|           Hockey|              

In [289]:
Find the most successful coach by event based on the number of medals won by their team

NameError: name 'l' is not defined

In [305]:
medalsDf.show()
CoachesDf.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [307]:
teamsDf.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [330]:
medalsDf.join(CoachesDf,medalsDf["TeamCountry"]==CoachesDf["Country"],"inner")\
.join(teamsDf,(teamsDf["Country"]==CoachesDf["Country"]) & (teamsDf["Discipline"]==CoachesDf["Discipline"]) & (teamsDf["Event"]==CoachesDf["Event"]),"inner") \
.orderBy(col("total").cast("int").desc())\
.show()

+----+--------------------+----+------+------+-----+-------------+--------------------+--------------------+-----------------+--------+-------------+-----------------+--------------------+--------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|                Name|             Country|       Discipline|   Event|     TeamName|       Discipline|             Country|   Event|
+----+--------------------+----+------+------+-----+-------------+--------------------+--------------------+-----------------+--------+-------------+-----------------+--------------------+--------+
|   1|United States of ...|  39|    41|    33|  113|            1|   ANDONOVSKI Vlatko|United States of ...|         Football|   Women|United States|         Football|United States of ...|   Women|
|   1|United States of ...|  39|    41|    33|  113|            1|          BERG Laura|United States of ...|Baseball/Softball|Softball|United States|Baseball/Softball|United States of ...|Softball|
|   1|Unit

In [342]:
print("medalsDf")
medalsDf.printSchema()
print("CoachesDf")
CoachesDf.printSchema()
print("teamsDf")
teamsDf.printSchema()


medalsDf
root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)

CoachesDf
root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)

teamsDf
root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [358]:
medals_df=medalsDf
coaches_df=CoachesDf.withColumnRenamed("Event","coachesEvent")
teams_df=teamsDf.withColumnRenamed("Event","teamEvent")


In [359]:
teams_medals_df = teams_df.join(medals_df, teams_df["Country"] == medals_df["TeamCountry"], "inner")

# Step 2: Join the result with Coaches on Discipline, Country, and Event
final_df = teams_medals_df.join(coaches_df, 
                                (teams_medals_df["Discipline"] == coaches_df["Discipline"]) &
                                (teams_medals_df["Country"] == coaches_df["Country"]) &
                                (teams_medals_df["teamEvent"] == coaches_df["coachesEvent"]),
                                "inner")
final_df.show()

+-------------+-----------------+--------------------+---------+----+--------------------+----+------+------+-----+-------------+--------------------+--------------------+-----------------+------------+
|     TeamName|       Discipline|             Country|teamEvent|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|                Name|             Country|       Discipline|coachesEvent|
+-------------+-----------------+--------------------+---------+----+--------------------+----+------+------+-----+-------------+--------------------+--------------------+-----------------+------------+
|    Australia|Artistic Swimming|           Australia|     Duet|   6|           Australia|  17|     7|    22|   46|            6|    MONTICO Loredana|           Australia|Artistic Swimming|        Duet|
|    Australia|Artistic Swimming|           Australia|     Team|   6|           Australia|  17|     7|    22|   46|            6|    MONTICO Loredana|           Australia|Artistic Swimming

In [363]:
final_df = final_df.withColumn("TotalMedals", col("Gold").cast("int") + col("Silver").cast("int") + col("Bronze").cast("int"))
window_spec = Window.partitionBy("coachesEvent").orderBy(col("TotalMedals").desc())
ranked_df = final_df.withColumn("Rank", rank().over(window_spec))

most_successful_coach_df = ranked_df.filter(col("Rank") == 1).select("Name", "coachesEvent", "TotalMedals")

# Show the result
most_successful_coach_df.show()

+-----------------+------------+-----------+
|             Name|coachesEvent|TotalMedals|
+-----------------+------------+-----------+
|  WEINSTEIN Jerry|    Baseball|        113|
|   FUENTES Andrea|        Duet|        113|
|   POPOVICH Gregg|         Men|        113|
|      FRIDAY Mike|         Men|        113|
|      SPERAW John|         Men|        113|
|   UDOVICIC Dejan|         Men|        113|
|     TARR Heather|    Softball|        113|
|   FLOWERS Tairia|    Softball|        113|
|      ERIKSEN Ken|    Softball|        113|
|    DOBSON Howard|    Softball|        113|
|       BERG Laura|    Softball|        113|
|         WANG Jie|        Team|         88|
|      STALEY Dawn|       Women|        113|
|ANDONOVSKI Vlatko|       Women|        113|
|      BROWN Chris|       Women|        113|
|     KIRALY Karch|       Women|        113|
|   KRIKORIAN Adam|       Women|        113|
+-----------------+------------+-----------+

